In [38]:
import pandas as pd
import seaborn as sb

In [169]:
data = pd.read_csv("assembly_stats.txt", sep='\t')

In [170]:
non_giab =data[(data["project"] != 'giab')]
#data = data[~( (data["project"] == 'hprc') & (data['sample'] == "HG002") )]

In [171]:
non_giab['n50'].describe().astype(int)

count         172
mean     35372147
std      14742684
min      14450795
25%      25739780
50%      30466158
75%      44413548
max      87239510
Name: n50, dtype: int64

In [172]:
data.groupby('project')['n50'].describe().T.astype(int)

project,eichler,giab,hprc,li
count,70,2,94,8
mean,27924723,161277649,42017870,22449875
std,6001634,1978570,16441231,4953928
min,14450795,159878589,17870629,14858698
25%,25651068,160578119,27560228,19033083
50%,28669490,161277649,41844848,22464944
75%,32757224,161977179,55038136,26383565
max,40327815,162676710,87239510,28262556


In [173]:
data[data['project'].isin(["eichler", "li"])]['n50'].describe().astype(int)

count          78
mean     27363200
std       6107628
min      14450795
25%      22114177
50%      28215538
75%      32113336
max      40327815
Name: n50, dtype: int64

In [174]:
data[data['sample'] == 'HG002'].groupby(['project'])['n50'].mean()

project
giab    161277649.5
hprc     86350596.5
Name: n50, dtype: float64

In [178]:
cov = pd.read_csv("coverage_stats.txt", sep='\t')
cov.head()

,mat,pat,mat_1x,pat_1x,diploid,auto,chrX,final,project,sample,sex,chrY,PARchrX
0,2802512086,2802845737,2789506233,2790710049,2809555401,2622223539,148338260,2770561799,li,HG00733,F,NaN,NaN
1,2804072988,2805462913,2792846762,2795559333,2815842213,2623624876,148858854,2772483730,eichler,HG03125,F,NaN,NaN
2,2842071604,2843282730,2820172774,2824059467,2834014412,2660552442,149665379,2810217821,hprc,HG02257,F,NaN,NaN
3,2718388169,2842479961,2695715244,2820423432,2855377588,2658249414,147563473,2826906412,hprc,HG03492,M,18773463.0,2320062.0
4,2844554217,2844178555,2820885125,2819807634,2837414524,2653228640,150043330,2803271970,hprc,HG01071,F,NaN,NaN


In [179]:
len(cov)

86

In [180]:
cov['sample'].nunique() # -1 because HG002 == NA24385

79

In [196]:
cov['overall_pct'] = cov['diploid'] / 2_923_715_986
cov['overall_pct'].describe()

count    86.000000
mean      0.967714
std       0.006665
min       0.954111
25%       0.962415
50%       0.969241
75%       0.970851
max       0.977883
Name: overall_pct, dtype: float64

In [181]:
cov['genome_pct'] = cov['final'] / 2_923_715_986 # Non-Ns of grch38 autosome X/Y

In [182]:
cov['genome_pct'].describe()

count    86.000000
mean      0.944750
std       0.025493
min       0.895440
25%       0.943032
50%       0.955738
75%       0.961125
max       0.970403
Name: genome_pct, dtype: float64

In [183]:
cov.groupby('project')[["genome_pct"]].describe()

genome_pct                                                    \
             count      mean       std       min       25%       50%   
project                                                                
eichler       35.0  0.923148  0.024054  0.895440  0.898191  0.941209   
hprc          47.0  0.962443  0.004744  0.953408  0.959373  0.960998   
li             4.0  0.925877  0.026230  0.903000  0.903258  0.925480   

                             
              75%       max  
project                      
eichler  0.946542  0.948274  
hprc     0.967293  0.970403  
li       0.948099  0.949547

In [185]:
data['haplotype'] = data['haplotype'].map({"H1":"pat", "H2":"mat", "1":"mat", "2":"pat", "mat":"mat", "pat":"pat"})
data

,sample,project,haplotype,n_contigs,n50
0,NA19983,eichler,pat,1465,35517528
1,HG01928,hprc,pat,244,54860574
2,HG01505,eichler,mat,2526,28512550
3,HG00096,eichler,mat,2185,28196602
4,HG00735,hprc,pat,244,57240366
...,...,...,...,...,...
169,HG01952,hprc,pat,307,55302156
170,HG01123,hprc,pat,340,55326480
171,HG01109,hprc,mat,473,30391630
172,HG01978,hprc,mat,286,53079156


In [186]:
data = data.sort_values(["sample", "haplotype", "project"])

In [188]:
h1 = data[data['haplotype'] == 'mat'].set_index(['sample', 'project']).drop(columns='haplotype')
h2 = data[data['haplotype'] == 'pat'].set_index(['sample', 'project']).drop(columns='haplotype')

In [189]:
asm_view = h1.join(h2, lsuffix="_maternal_hap", rsuffix="_paternal_hap2")

In [190]:
asm_view

n_contigs_maternal_hap  n50_maternal_hap  \
sample  project                                             
HG00096 eichler                    2185          28196602   
HG00171 eichler                    2482          29266314   
HG002   giab                        355         162676710   
        hprc                        610          86209598   
HG00438 hprc                        276          49138847   
...                                 ...               ...   
NA20847 eichler                    1903          27719882   
NA21309 hprc                        608          17870629   
NA24385 eichler                    7639          25653134   
        li                         5735          24836347   
PGP1    li                         8582          18444923   

                 n_contigs_paternal_hap2  n50_paternal_hap2  
sample  project                                              
HG00096 eichler                     2263           28826430  
HG00171 eichler                     2452           29317192  
HG002   giab                         514          159878589  
        hprc                         445           86491595  
HG00438 hprc                         258           56346258  
...                                  ...                ...  
NA20847 eichler                     1953           28983715  
NA21309 hprc                         501           20876992  
NA24385 eichler                     8003           32602800  
        li                          5912           25830230  
PGP1    li                          8396           14858698  

[87 rows x 4 columns]

In [191]:
cov.loc[cov['sample'] == "HG002", "project"] = 'giab'

In [192]:
cov['project'].value_counts()

hprc       46
eichler    35
li          4
giab        1
Name: project, dtype: int64

In [193]:
asm_view.loc['HG002']

,n_contigs_maternal_hap,n50_maternal_hap,n_contigs_paternal_hap2,n50_paternal_hap2
project,,,,
giab,355,162676710,514,159878589
hprc,610,86209598,445,86491595


In [194]:
cov.set_index(["sample", "project"], inplace=True)

In [195]:
asm_view.join(cov).sort_index(level=1).reset_index().to_csv("final_stats.txt", sep='\t', index=False)